In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [4]:
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mrakovo,RU,52.72,56.62,43.23,77,63,4.76,broken clouds
1,1,Rikitea,PF,-23.12,-134.97,72.07,82,100,12.44,overcast clouds
2,2,Namatanai,PG,-3.67,152.43,84.76,69,100,9.08,light rain
3,3,Broken Hill,AU,-31.95,141.43,62.60,42,0,5.82,clear sky
4,4,Punta Arenas,CL,-53.15,-70.92,39.20,86,100,6.93,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Namatanai,PG,-3.67,152.43,84.76,69,100,9.08,light rain
11,11,Vaini,TO,-21.20,-175.20,75.20,64,40,16.11,scattered clouds
13,13,Bonthe,SL,7.53,-12.51,76.93,88,100,4.59,overcast clouds
15,15,Atambua,ID,-9.11,124.89,83.37,42,0,1.39,clear sky
19,19,Atuona,PF,-9.80,-139.03,78.60,74,16,21.43,few clouds
21,21,Kapaa,US,22.08,-159.32,84.20,66,75,9.17,broken clouds
22,22,Georgetown,MY,5.41,100.34,80.60,100,20,3.36,haze
28,28,Cabo San Lucas,MX,22.89,-109.91,89.01,62,5,5.82,clear sky
29,29,Butaritari,KI,3.07,172.79,82.96,74,87,10.71,overcast clouds
35,35,Brownwood,US,31.71,-98.99,78.80,69,1,6.93,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                269
City                   269
Country                269
Lat                    269
Lng                    269
Max Temp               269
Humidity               269
Cloudiness             269
Wind Speed             269
Current Description    269
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Namatanai,PG,-3.67,152.43,84.76,69,100,9.08,light rain
11,11,Vaini,TO,-21.20,-175.20,75.20,64,40,16.11,scattered clouds
13,13,Bonthe,SL,7.53,-12.51,76.93,88,100,4.59,overcast clouds
15,15,Atambua,ID,-9.11,124.89,83.37,42,0,1.39,clear sky
19,19,Atuona,PF,-9.80,-139.03,78.60,74,16,21.43,few clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

In [10]:
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Namatanai,PG,84.76,light rain,-3.67,152.43,
11,Vaini,TO,75.20,scattered clouds,-21.20,-175.20,
13,Bonthe,SL,76.93,overcast clouds,7.53,-12.51,
15,Atambua,ID,83.37,clear sky,-9.11,124.89,
19,Atuona,PF,78.60,few clouds,-9.80,-139.03,
21,Kapaa,US,84.20,broken clouds,22.08,-159.32,
22,Georgetown,MY,80.60,haze,5.41,100.34,
28,Cabo San Lucas,MX,89.01,clear sky,22.89,-109.91,
29,Butaritari,KI,82.96,overcast clouds,3.07,172.79,
35,Brownwood,US,78.80,clear sky,31.71,-98.99,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [12]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        #print("Hotel not found... skipping.")
        pass
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
print(hotel_df.head())

-----------------------------
Data Retrieval Complete      
-----------------------------
         City Country  Max Temp Current Description    Lat     Lng  \
2   Namatanai      PG     84.76          light rain  -3.67  152.43   
11      Vaini      TO     75.20    scattered clouds -21.20 -175.20   
13     Bonthe      SL     76.93     overcast clouds   7.53  -12.51   
15    Atambua      ID     83.37           clear sky  -9.11  124.89   
19     Atuona      PF     78.60          few clouds  -9.80 -139.03   

                               Hotel Name  
2                         Namatanai Hotel  
11                    Keleti Beach Resort  
13                 Bonthe Holiday Village  
15  Pondok Pesantren Al Muhajirin Atambua  
19                            Villa Enata  


In [13]:
hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Namatanai,PG,84.76,light rain,-3.67,152.43,Namatanai Hotel
11,Vaini,TO,75.20,scattered clouds,-21.20,-175.20,Keleti Beach Resort
13,Bonthe,SL,76.93,overcast clouds,7.53,-12.51,Bonthe Holiday Village
15,Atambua,ID,83.37,clear sky,-9.11,124.89,Pondok Pesantren Al Muhajirin Atambua
19,Atuona,PF,78.60,few clouds,-9.80,-139.03,Villa Enata
21,Kapaa,US,84.20,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
22,Georgetown,MY,80.60,haze,5.41,100.34,Cititel Penang
28,Cabo San Lucas,MX,89.01,clear sky,22.89,-109.91,Hotel Tesoro Los Cabos
29,Butaritari,KI,82.96,overcast clouds,3.07,172.79,Isles Sunset Lodge
35,Brownwood,US,78.80,clear sky,31.71,-98.99,La Quinta Inn & Suites by Wyndham Brownwood


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df =hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Namatanai,PG,84.76,light rain,-3.67,152.43,Namatanai Hotel
11,Vaini,TO,75.20,scattered clouds,-21.20,-175.20,Keleti Beach Resort
13,Bonthe,SL,76.93,overcast clouds,7.53,-12.51,Bonthe Holiday Village
15,Atambua,ID,83.37,clear sky,-9.11,124.89,Pondok Pesantren Al Muhajirin Atambua
19,Atuona,PF,78.60,few clouds,-9.80,-139.03,Villa Enata
...,...,...,...,...,...,...,...
711,Conceicao Das Alagoas,BR,77.00,clear sky,-19.91,-48.39,Bouganville Hotel
714,Aden,YE,83.61,overcast clouds,12.78,45.04,Sama Emirate Hotel
717,Hoi An,VN,87.80,few clouds,15.88,108.33,Belle Maison Hadana Hoi An Resort & Spa manage...
719,Austin,US,77.00,light rain,30.27,-97.74,The Driskill - In the Unbound Collection by Hyatt


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [17]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
# Convert into an array
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))